In [1]:
%run analyze-dict.py info ../ignoredFiles/Large-Sample-Dict/news-nGramDict.json


Loading from JSON file ...
Analyzing Dict: |██████████████████████████████| 100.0% Complete



In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from numpy import nan as NA
import json,os

In [13]:
def prepareDF(freqDict):
    n=len(freqDict)
    freqDF=DataFrame(freqDict)
    freqDF=freqDF.drop(n,axis=1)
    colname=[]
    for i in list(range(1,n)):
        colname.append('N'+str(i)+'-gram')
    freqDF.columns=colname
    freqDF.reset_index(inplace=True)
    return freqDF

In [14]:
freqDF=prepareDF(freqDict)

In [15]:
def estimateZr(freqDF,ngram):
    freqDF1=freqDF[['index',ngram]].dropna()
    freqDF1.columns=['r','Nr']
    freqDF1['q']=0
    freqDF1['t']=0
    freqDF1=freqDF1.reset_index(drop=True)
    last_index=0
    for i in range(len(freqDF1)):
        freqDF1.ix[i,'q']=last_index
        last_index=freqDF1.ix[i,'r']
    trailing_index=2*freqDF1.ix[len(freqDF1)-1,'r']-freqDF1.ix[len(freqDF1)-1,'q']
    for i in range(len(freqDF1)-1,-1,-1):
        freqDF1.ix[i,'t']=trailing_index
        trailing_index=freqDF1.ix[i,'r']
    freqDF1['Zr']=freqDF1['Nr']/(0.5*freqDF1['t']-0.5*freqDF1['q'])
    x=np.array(freqDF1['r'])
    y=np.array(freqDF1['Zr'])
    x_log=np.log(x)
    y_log=np.log(y)
    c=np.polyfit(x_log,y_log,1)
    newKey=DataFrame(list(range(1,freqDF1['r'].max()+2)))
    newKey.columns=['r']
    mergedDF=pd.merge(newKey,freqDF1,on='r',how='left')[['r','Zr']]
    mergedDF=mergedDF.fillna(0)
    mergedDF['needEstimate']=True
    mergedDF['needEstimate']=mergedDF['needEstimate'].where(mergedDF['Zr']!=0,False)
    mergedDF['Zr']=mergedDF['Zr'].where(mergedDF['Zr']!=0,np.exp(c[1]+c[0]*np.log(mergedDF['r'])))
    return mergedDF

In [16]:
def GTdiscount(freqDF):
    n=len(freqDict)
    colname=[]
    GTEstDict={}
    for i in list(range(1,n)):
        colname.append('N'+str(i)+'-gram')
    print('Calculating Good-Turing Discount Ratio for')
    for ngram in colname:     
        print(ngram,'......')
        GTEstDict[ngram]=estimateZr(freqDF,ngram)
        for i in GTEstDict[ngram][GTEstDict[ngram]['needEstimate']==True]['r']:
            GTEstDict[ngram].ix[i-1,'r-est']=(i+1)*GTEstDict[ngram].ix[i,'Zr']/GTEstDict[ngram].ix[i-1,'Zr']
        GTEstDict[ngram]=GTEstDict[ngram][GTEstDict[ngram]['needEstimate']==True][['r','r-est']]
        GTEstDict[ngram]['d']=GTEstDict[ngram]['r-est']/GTEstDict[ngram]['r']
        GTEstDict[ngram]['d'][GTEstDict[ngram]['d']>=1]=0.999
        Temp=GTEstDict[ngram][['r','d']]
        GTEstDict[ngram]={}
        GTEstDict[ngram]['ct']=Temp['r'].tolist()
        GTEstDict[ngram]['d']=Temp['d'].tolist()
    return GTEstDict   

In [ ]:
GTEstDict=GTdiscount(freqDF)

Calculating Good-Turing Discount Ratio for
N1-gram ......


In [8]:
GTEstDict

{'1-gram': {'ct': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   1

In [10]:
savetopath='/home/yewenhe0904/Developing/ds-capstone-project/ignoredFiles/news_GT_Discount.json'
f1=open(savetopath,"w")
json.dump(GTEstDict,f1)
